In [1]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 1000
Na = 3
Ns = Nx * Ny

In [41]:
import itertools

from scipy import interpolate

import sys

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
#       opt_thinn_end = tau[0, k]
        opt_thinn_end = tau[1, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    tau_1d = np.log10(tau.T).reshape(Nz * Ns)
    
    delta = np.abs(np.diff(tau_1d))
    
    idx_min = np.argmin(np.abs(delta - 0.001))
            
    return delta[idx_min]

def const_interpol_grid(tau, Nz, Ns):
    
    thin, thick = find_interpol_range(tau, Nz, Ns)
    delta =       find_shortest_delta(tau, Nz, Ns)
    
    Ng = int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(i, target, tau, T, igrid, Ns):
    
    T_mean = np.zeros(len(igrid))
    
    for k in tqdm(range(Ns), desc = 'interpolate and average ' + target):
    
        f = interpolate.interp1d(np.log10(tau[:, k]), T[:, k])
        
        Ti = f(igrid)
        
        T_mean += Ti
        
    T_mean /= Ns
    
    return T_mean

In [42]:
mu_id = ['10', '6', '2']

T_ssd =    np.ones((Na, Nz, Ns)) * np.nan
t200_ssd = np.ones((Na, Nz, Ns)) * np.nan

T_mag =    np.ones((Na, Nz, Ns)) * np.nan
t200_mag = np.ones((Na, Nz, Ns)) * np.nan

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_mag = np.ones((Na, 110000)) * np.nan

Ti_ssd =   np.ones((Na, 110000)) * np.nan
Ti_mag =   np.ones((Na, 110000)) * np.nan

for i in range(Na):
    
    print("---------------")
    print('mu = ', mu_id[i])
    print("---------------")
    
    if i == 0:           mu_mod = mu_id[i]
    
    if i == 1 or i == 2: mu_mod = '0' + mu_id[i]    
    
    print('SSD cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    print('reading tau...')
    
    t200 = np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/result_ttau200.222809.nc')['tau'])
    
    Nz_cur = len(t200[:, 0, 0])
    
    t200_ssd[i, : Nz_cur, :] = t200.reshape(Nz_cur, Ny * Nx)
    
    if i == 1 or i == 2: t200_ssd[i, 0, :] = 1e-15
    
    print('sampling tau...')
    
    tau = t200_ssd[i, : Nz_cur, sample].T
    
    grid = const_interpol_grid(tau, Nz_cur, Ns)
    
    grid_ssd[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    
    T = np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/T_mu_' + mu_id[i] + '.222809.nc')['T'])
    
    T_ssd[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)
    
    T_mean = interpol_and_average(i, 'SSD temp', tau, T_ssd[i, : Nz_cur, sample].T, grid, Ns)

    Ti_ssd[i, : len(T_mean)] = T_mean
    
    print('300G cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    print('reading tau...')
    
    t200 = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/result_ttau200.627321.nc')['tau'])
    
    Nz_cur = len(t200[:, 0, 0])
    
    t200_mag[i, : Nz_cur, :] = t200.reshape(Nz_cur, Ny * Nx)
    
    if i == 1 or i == 2: t200_mag[i, 0, :] = 1e-15
    
    print('sampling tau...')
    
    tau = t200_mag[i, : Nz_cur, sample].T
    
    grid = const_interpol_grid(tau, Nz_cur, Ns)
    
    grid_mag[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    
    T = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/T_mu_' + mu_id[i] + '.627321.nc')['T'])
    
    T_mag[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)
    
    T_mean = interpol_and_average(i, '300G temp', tau, T_mag[i, : Nz_cur, sample].T, grid, Ns)

    Ti_mag[i, : len(T_mean)] = T_mean

---------------
mu =  10
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


interpolate and average SSD temp: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:32<00:00, 8030.46it/s]


300G cube
reading tau...
sampling tau...
reading temperature...


interpolate and average 300G temp: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:32<00:00, 7945.40it/s]


---------------
mu =  6
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


...

interpolate and average 300G temp: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:35<00:00, 7313.83it/s]


---------------
mu =  2
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


interpolate and average SSD temp: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:43<00:00, 6025.16it/s]


300G cube
reading tau...
sampling tau...
reading temperature...


interpolate and average 300G temp: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:44<00:00, 5886.06it/s]


In [ ]:
np.savez('tau200_rot', t200_ssd = t200_ssd, T_ssd = T_ssd, t200_mag = t200_mag, T_mag = T_mag)

In [ ]:
t200_ssd = np.load('tau200_rot.npz')['t200_ssd']
t200_mag = np.load('tau200_rot.npz')['t200_mag']

T_ssd = np.load('tau200_rot.npz')['T_ssd']
T_mag = np.load('tau200_rot.npz')['T_mag']

In [ ]:
np.savez('tau200_rot_interpol', grid_ssd = grid_ssd, Ti_ssd = Ti_ssd,\
                                grid_mag = grid_mag, Ti_mag = Ti_mag)

In [3]:
grid_ssd = np.load('tau200_rot_interpol.npz')['grid_ssd']
grid_mag = np.load('tau200_rot_interpol.npz')['grid_mag']

Ti_ssd =   np.load('tau200_rot_interpol.npz')['Ti_ssd']
Ti_mag =   np.load('tau200_rot_interpol.npz')['Ti_mag']

In [4]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.6', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(5, 11.25 * 2 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(2, 2, width_ratios = [1, 1], hspace = 0.2, wspace = 0.3)

tem_ssd = plt.subplot(gs[0, 0])
t_ssd_c = plt.subplot(gs[0, 1])

tem_mag = plt.subplot(gs[1, 0])
t_mag_c = plt.subplot(gs[1, 1])

for i in range(len(mus)):

    tem_ssd.plot(grid_ssd[i, :], Ti_ssd[i, :], label = r'$\mu = $' + mus[i])
    t_ssd_c.plot(grid_ssd[i, :], Ti_ssd[i, :])
    
    tem_mag.plot(grid_mag[i, :], Ti_mag[i, :])
    t_mag_c.plot(grid_mag[i, :], Ti_mag[i, :])
    
    tem2_ssd = t_ssd_c.twinx()
    tem2_mag = t_mag_c.twinx()
    
    tem2_ssd.yaxis.set_ticks([])
    tem2_mag.yaxis.set_ticks([])

    tem_ssd.set_xscale('log')
    t_ssd_c.set_xscale('log')
    tem_mag.set_xscale('log')
    t_mag_c.set_xscale('log')

    tem_ssd.set_xlim(1e+5, 1e-1)
    tem_mag.set_xlim(tem_ssd.get_xlim())

    t_ssd_c.set_xlim(1e+2, 1e-1)
    t_mag_c.set_xlim(t_ssd_c.get_xlim())

    tem_ssd.set_ylim(2500, 12000)
    tem_mag.set_ylim(2500, 12000)
    
    t_ssd_c.set_ylim(2950, 4050)
    t_mag_c.set_ylim(3000, 4500)

    tem_ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    tem_mag.yaxis.set_minor_locator(AutoMinorLocator(4))

    tem_ssd.set_ylabel('Temperature, K')
    tem_mag.set_ylabel('Temperature, K')
    
    tem2_ssd.set_ylabel('SSD')
    tem2_mag.set_ylabel('300G')
    
    tem_mag.set_xlabel(r'$\tau_\mathrm{200}$')
    t_mag_c.set_xlabel(r'$\tau_\mathrm{200}$')
    
    tem_ssd.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('mean_temp_of_tau200.pdf', bbox_inches = 'tight')

plt.close('all')

In [54]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.6', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25 * 3 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(3, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.2)

for i in tqdm(range(3)):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
    for k in np.random.choice(Nx * Ny, 16, replace = False):
#    for k in range(Ns):
        
        ssd.plot(t200_ssd[i, :, k], T_ssd[i, :, k], color = 'gray', linewidth = 0.5)
        mag.plot(t200_mag[i, :, k], T_mag[i, :, k], color = 'gray', linewidth = 0.5)
        
    ssd.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k')
    mag.plot(grid_mag[i, :], Ti_mag[i, :], color = 'r')
    
    com.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k', label = 'SSD')
    com.plot(grid_mag[i, :], Ti_mag[i, :], color = 'r', label = '300G')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(2e+5, 1e-3)
    ssd.set_ylim(2000, 13500)
    
    ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    ssd.set_ylabel('Temperature, K')
    
    if i == 0: ssd.set_title('SSD')
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    mag.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    if i == 0: mag.set_title('300G')
        
    com.set_xscale('log')
    
    com.set_xlim(ssd.get_xlim())
    com.set_ylim(ssd.get_ylim())
    
    com.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    if i == 2: ssd.set_xlabel(r'$\tau_\mathrm{200}$')
    if i == 2: mag.set_xlabel(r'$\tau_\mathrm{200}$')
    if i == 2: com.set_xlabel(r'$\tau_\mathrm{200}$')
    
    com.set_ylabel(r'$\mu =$' + mus[i])
    com.yaxis.set_label_position("right")
    
    if i == 0: com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2_tau200.pdf', bbox_inches = 'tight')

plt.close('all')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.14it/s]
